In [ ]:
import re
import pandas as pd
import bikeSeoul as bs
import distance_kakaoapi as kakao

from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

In [ ]:
app_token = #slack-app-token(soket)
bot_token = #slack-bot-token
app = App(token= bot_token)
location = []
count = 1
@app.event("app_mention")  # 앱을 언급했을 때
def who_am_i(event, client, message, say):
    global count, a,b
    if count == 1:
        say(f'안녕하세요! <@{event["user"]}>님 무엇을 도와 드릴까요?')
        count+=1
    else:
        #사용자의 입력 값 변환
        user_text = event['text']
        user_text = re.sub('[<>]*@[A-Za-z0-9]+[<>]*','',user_text)
        #사용자 위치 알기
        location = [user_text.strip()]
        df = kakao.keywords(location)
        df = df.reset_index()
        a,b = kakao.make_map(df)
        my_location = (float(a),float(b))
        
        #사용자 위치 근처 따릉이 3곳
        s = bs.create_api()
        df_test = bs.latilong(s,my_location)
        df_total_distance = bs.bestdistance(df_test,s)
        bs.make_map2(df_total_distance)
        
        ### 근처 따릉이 대여소 거리 계산
        for text_station,distan in df_total_distance[['stationName','distance']].values:
            text_station = re.sub('[0-9]+\.','',text_station).strip()
            distan = round(float(distan),3)
            say(f'근처 따릉이 대여소는 {text_station}입니다.')
            say(f'대여소까지 직선 거리는 {distan}m 입니다.')
            
        #html -> 사진 변환 + 사진 전송
        path = bs.picture()
        client.files_upload_v2(channels = event['channel'], file = path)
        
        
        
@app.event("message")
def handle_message_events(body, logger):
    logger.info(body)
    
if __name__ == '__main__':
    SocketModeHandler(app, app_token).start()
    
    
#경 / 위도 별 날씨
# 상대 경로로 바꾸기 
# 내 위치까지 마커로 표시하기